# Preamble

## Variables

In [1]:
DATA_AMOUNT = [10, 100]
TRAIN_SPLIT = 0.8

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import get_text
import ceaser

## Creating Data

In [4]:
trainData, testData = get_text.initialise(ceaser.encrypt, *DATA_AMOUNT, TRAIN_SPLIT, stream=True)

Using the latest cached version of the dataset since manu/project_gutenberg couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Tristan\.cache\huggingface\datasets\manu___project_gutenberg\default\0.0.0\164853d214065df26a630ee1ab91a0c39e461caf (last modified on Thu Sep  4 19:18:40 2025).


  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
trainData

[['sepd lkla ydwlpan reee pda hwop uawno kb deo heba ydwlpan et pda cajeqo kb wzzeokj wzzeokj ydwlpan e pda opwpa kb ajcheod okyeapu wjz happano wbpan pda naopknwpekj kb pda bkqn ajcheod iaj kb happano sdkoa snepejco ikop bqhhu aixkzu pda olenep kb pda aecdpaajpd yajpqnu pda kja sdk lnkrezao pda xekcnwldan sepd pda oywjpeaop iwpanewho eo wzzeokj ej deo fkqnjwh pk opahhw deo okyewh ranoao wjz deo happano pk deo bneajzo sa dwra w rerez leypqna kb pdkoa nahwpekjo sepd skiaj wjz pdwp lnkpnwypaz oqbbanejc sdeyd ejraop sepd oqyd pnwcey ejpanaop pda deopknu kb osebp lkla xu pda',
  22],
 ['e v nthjiyn fzvon ndyizt xjgqdi xvmgtgz ejci idxcjg hj xgjoc xzion kzm qjgphz joczm qjgphzn di kmzkvmvodji kpwgdnczy wt cvmkzm wmjoczmn izr tjmf vit ja ocz vwjqz rjmfn rdgg wz nzio wt hvdg kjnovbz kmzkvdy oj vit kvmo ja ocz pidozy novozn xvivyv jm hzsdxj ji mzxzdko ja ocz kmdxz xjiozion kvbz xcvkozm d ocz novoz ja zibgdnc njxdzot viy gzoozmn vaozm ocz mznojmvodji xcvkozm dd vyydnji n avhdgt viy zypxvodji xc